# Aufgabe 2 Wein

## Data Cleaning

In [39]:
%pip install regex
import pandas as pd
import numpy as np
import regex as re



Note: you may need to restart the kernel to use updated packages.


In [40]:
df = pd.read_csv('wein.csv')
df = df.dropna()
df = df.drop_duplicates()
# convert all columns to strings to handle leading zeros and punctuation
df = df.applymap(str)
# drop the punct at the end of the string/number
df = df.applymap(lambda x: x.rstrip('.') if isinstance(x, str) else x)
# drop the 0 in the beginning of the string/number
df = df.applymap(lambda x: re.sub(r'^0+(?=\d)','',x) if isinstance(x, str) else x)
# convert columns back to appropriate numeric types
df = df.apply(pd.to_numeric, errors='coerce')
print(df.dtypes)
df
#df.to_csv('wein_clean.csv', index=False)
df.head()



Alkohol                    float64
Apfelsaeure                float64
Asche                      float64
Aschen_Alkanitaet          float64
Magnesium                    int64
Alle_Phenole               float64
Flavanoide                 float64
Nichtflavanoide_Phenole    float64
Proanthocyanide            float64
Farbintensitaet            float64
Farbwert                   float64
Proteinwert                float64
Prolinwert                   int64
dtype: object


/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_67670/3946539532.py:5: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_67670/3946539532.py:7: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_67670/3946539532.py:9: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,Alkohol,Apfelsaeure,Asche,Aschen_Alkanitaet,Magnesium,Alle_Phenole,Flavanoide,Nichtflavanoide_Phenole,Proanthocyanide,Farbintensitaet,Farbwert,Proteinwert,Prolinwert
0,14.23,1.71,2.43,15.06,127,2.08,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.02,1.78,2.14,11.02,100,2.65,2.76,0.26,1.28,4.38,1.05,3.04,1050
2,13.16,2.36,2.67,18.06,101,2.08,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.05,16.08,113,3.85,3.49,0.24,2.18,7.08,0.86,3.45,1480
4,13.24,2.59,2.87,21.00,118,2.08,2.69,0.39,1.82,4.32,1.04,2.93,735


In [41]:
#df.info()
#df.describe()

## Linear Regression

### Quality values of the regression (R^2 score)
The R² score, also known as the coefficient of determination, is a statistical measure that indicates how well the independent variables explain the variability of the dependent variable in a regression model. It provides an indication of the goodness of fit of the model.  
- R² = 1: Indicates that the regression model perfectly explains the variability of the dependent variable.
- R² = 0: Indicates that the regression model does not explain any of the variability of the dependent variable.
- R² < 0: In some cases, R² can be negative, indicating that the model is worse than a horizontal line (mean of the dependent variable).

In [42]:
#sns.pairplot(df, kind = 'scatter')

In [43]:
#sns.lmplot(x='Proanthocyanide', y='Flavanoide', data =df, scatter_kws={'alpha':0.3})

## Clustering
### k-means algorithm
### evaluation


# Dash App

In [ ]:
%pip install scikit-learn dash plotly

import pandas as pd
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

data = df

# Standardization scales the data so that all features contribute equally to the clustering process, especially when features have different ranges.
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Regressionen berechnen
def calculate_regressions(data):
    '''
    for each pair of columns in the data, calculate the R² of a linear regression
    param data: a pandas DataFrame
    return: a pandas DataFrame with columns 'X', 'Y', 'R²'
    '''
    dic_list = [
        {'X': x, 'Y': y, 'R^2': r2_score(data[y], LinearRegression().fit(data[[x]], data[y]).predict(data[[x]]))}
        for x, y in combinations(data.columns, 2)
    ]
    return pd.DataFrame(dic_list)

regression_results = calculate_regressions(data)

# Dash App 
app = Dash(__name__)
app.layout = html.Div([

    # 1. Linear Regression
    html.Div([
        html.H2("Lineare Regression", style={'textAlign': 'center'}),
        
        # Dropdown für die Auswahl von R^2
        dcc.Dropdown(
            id='r2-threshold-dropdown',
            options=[
                {'label': 'R² > 0.1', 'value': 0.1},
                {'label': 'R² > 0.3', 'value': 0.3},
                {'label': 'R² > 0.5', 'value': 0.5},
                {'label': 'R² > 0.7', 'value': 0.7}, 
            ],
            value=0.3,  
            style={'width': '50%'}
        ),
        
        # Dropdown for selecting regression pairs
        dcc.Dropdown(id='xy-pairs',value='Alle_Phenole,Flavanoide',placeholder="Wähle ein Paar"),
        
        # Graph für die  Regression
        dcc.Graph(id='regression-plot'),
        
        html.Div(id='regression-stats')
    ]),

    # 2. KMeans Clustering
    html.Div([
        html.H2("KMeans Clustering", style={'textAlign': 'center'}),
        dcc.Dropdown(
            id='kmeans-dropdown',
            options=[
                {'label': 'K=2', 'value': 2},
                {'label': 'K=3', 'value': 3},
                {'label': 'K=4', 'value': 4},
                {'label': 'K=5', 'value': 5},
            ],
            value=2,
        ),
        # 2.1 Graph for visualizing the KMeans Clustering
        dcc.Graph(id='cluster-graph'),
        # 2.2 Graph for evaluating metrics
        dcc.Graph(id='evaluation-graph'),
    ])

])

# 1. Linear Regression
@app.callback(
    [Output('xy-pairs', 'options'), Output('regression-plot', 'figure'), Output('regression-stats', 'children')],
    Input('r2-threshold-dropdown', 'value'),
    Input('xy-pairs', 'value')
)
def update_content(r2_threshold, selected_pair):
    # Filter für den Dropdown nach R²
    filtered = regression_results[regression_results['R^2'] > r2_threshold]
    options = [{'label': f"{row['X']} und {row['Y']}", 'value': f"{row['X']},{row['Y']}"} for _, row in filtered.iterrows()]

    #default_option = {'label': 'Alle_Phenole und Flavonoide', 'value': 'Alle_Phenole,Flavanoide'}

    if not selected_pair:
        return options, {}, "ein Paar auswählen."
    
    x, y = selected_pair.split(',')
    X, y_vals = data[[x]].values, data[y].values
    model = LinearRegression().fit(X, y_vals)
    r2, slope, intercept = r2_score(y_vals, model.predict(X)), model.coef_[0], model.intercept_

    # Plot erstellen
    fig = px.scatter(x=X.flatten(), y=y_vals, labels={'x': x, 'y': y})
    fig.add_scatter(x=X.flatten(), y=model.predict(X), mode='lines', name='Regressionslinie')

    # Statistiken
    stats = f"R²: {r2:.4f}, Steigung: {slope:.4f}, Achsenabschnitt: {intercept:.4f}"
    return options, fig, stats


# 2. KMeans Clustering
@app.callback(
    [Output('cluster-graph', 'figure'), Output('evaluation-graph', 'figure')],
    [Input('kmeans-dropdown', 'value')]
)
def update_clustering(n_clusters):

    # functions for clustering, it returns
    def perform_kmeans(data, n_clusters):

        # KMeans Algorithm (k-means++ variant): The algorithm initializes the cluster centers in a smart way to speed up convergence. 
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, init='k-means++')

        # Cluster Labels: Assigns each data point to a cluster
        labels = kmeans.fit_predict(data)

        # return labels and kmeans object, notice that kmeans is a object that contains the cluster centers, and other information
        return labels, kmeans

    def evaluate_clustering(data, labels):
        # Silhouette Score: The silhouette score measures how similar an object is to its own cluster compared to other clusters. It returns Mean Silhouette Coefficient for all samples. By defaul using euclidean distance.
        silhouette_avg = silhouette_score(data, labels) # the higher the better
        # Davies-Bouldin Index: The Davies-Bouldin index is defined as the average similarity measure of each cluster with its most similar cluster. It returns the average index of all clusters.
        davies_bouldin_avg = davies_bouldin_score(data, labels) # the lower the better
        return silhouette_avg, davies_bouldin_avg

    
    # PCA for visualization: For high-dimensional data, we reduce the data to 2D using PCA for visualization.
    pca = PCA(n_components=2)
    data_pca = pca.fit_transform(data_scaled)

    # Perform clustering with selected number of clusters
    labels, kmeans = perform_kmeans(data_scaled, n_clusters)
    
    # Evaluate metrics for the chosen number of clusters
    silhouette_avg, davies_bouldin_avg = evaluate_clustering(data_scaled, labels)

    # center of the clusters in the PCA space
    cluster_centers_pca = pca.transform(kmeans.cluster_centers_)

    # Create scatter plot for clusters
    cluster_fig = px.scatter(
        x=data_pca[:, 0], y=data_pca[:, 1],
        color=labels.astype(str),
        title=f'K-Means Clustering with {n_clusters} Clusters',
        labels={'x': 'PCA1', 'y': 'PCA2'}
    )
    cluster_fig.update_layout(coloraxis_colorbar=dict(title='Cluster'))


    # Add cluster centers to the scatter plot
    for i, (x, y) in enumerate(cluster_centers_pca):
        cluster_fig.add_trace(go.Scatter(
            x=[x], y=[y],
            mode='markers',
            marker=dict(color='red', size=12, symbol='star'),
            name=f'Center {i + 1}'
        ))

    # Evaluate metrics for all cluster configurations
    cluster_range = range(2, 6)
    silhouette_scores = [] # list to store silhouette scores
    davies_bouldin_scores = [] # list to store davies bouldin scores
    
    for k in cluster_range:
        labels, _ = perform_kmeans(data_scaled, k)
        silhouette_avg, davies_bouldin_avg = evaluate_clustering(data_scaled, labels)
        silhouette_scores.append(silhouette_avg)
        davies_bouldin_scores.append(davies_bouldin_avg)
    
    # Create evaluation metrics plot
    evaluation_fig = go.Figure()
    evaluation_fig.add_trace(go.Scatter(
        x=list(cluster_range), y=silhouette_scores,
        mode='lines+markers', name='Silhouette Coefficient'
    ))
    evaluation_fig.add_trace(go.Scatter(
        x=list(cluster_range), y=davies_bouldin_scores,
        mode='lines+markers', name='Davies-Bouldin Index'
    ))
    evaluation_fig.update_layout(
        title='Evaluation Metrics for K-Means Clustering',
        xaxis_title='Number of Clusters',
        yaxis_title='Score',
        legend_title='Metrics'
    )
    
    return cluster_fig, evaluation_fig

if __name__ == '__main__':
    app.run_server(debug=True)





Note: you may need to restart the kernel to use updated packages.
